<a href="https://colab.research.google.com/github/it21023996/machine-learning/blob/main/Movie_Recommendation_word2Veccosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('/content/top10K-TMDB-movies.csv')

In [8]:
df.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [9]:
df.shape

(10000, 9)

In [10]:
df1 = df[['id','title','genre','overview']]

In [11]:
df1.head()

,id,title,genre,overview
0,278,The Shawshank Redemption,"Drama,Crime",Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,"Drama,History,War",The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,"Drama,Crime",In the continuing saga of the Corleone crime f...


In [12]:
df1.isnull().sum()

,0
id,0
title,0
genre,3
overview,13


In [13]:
df1_cleaned = df1[df1['overview'].notna()]

In [14]:
df1_cleaned = df1_cleaned.reset_index(drop=True)

In [15]:
df1_cleaned.isnull().sum()

,0
id,0
title,0
genre,2
overview,0


In [16]:
df1_cleaned.shape

(9987, 4)

##data preprocessing

In [17]:
from gensim.models import Word2Vec

In [18]:
import re

In [19]:
def preprocess(text):
  text = str(text).lower()
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  tokens = text.split()
  return tokens

In [20]:
df1_cleaned['token'] = df1_cleaned['overview'].apply(preprocess)

In [21]:
w2vec_model = Word2Vec(sentences = df1_cleaned['token'],vector_size=100,window=5,min_count=1,workers=4)

In [25]:
import numpy as np

In [26]:
def get_movie_vector(tokens):
    vecs = []
    for word in tokens:
        if word in w2vec_model.wv:
            vecs.append(w2vec_model.wv[word])
    if len(vecs) == 0:
        return np.zeros(100)  # fallback vector
    return np.mean(vecs, axis=0)

In [28]:
df1_cleaned['vector'] = df1_cleaned['token'].apply(get_movie_vector)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
def recommend_by_text(input_text, top_n=10):
    # Preprocess input
    tokens = preprocess(input_text)

    # Get input vector
    input_vec = get_movie_vector(tokens).reshape(1, -1)

    # Stack all movie vectors into a matrix
    movie_matrix = np.vstack(df1_cleaned['vector'].values)

    # Compute cosine similarity
    similarities = cosine_similarity(input_vec, movie_matrix).flatten()

    # Get top N indices
    top_indices = similarities.argsort()[::-1][:top_n]

    # Return top N movie titles
    return df1_cleaned.iloc[top_indices]['title'].tolist()

In [36]:
recommend_by_text("A man is wrongfully imprisoned for a crime he didn’t commit and struggles to survive in jail", top_n=5)

['Chouchou', 'My Sassy Girl', 'Masculin Féminin', 'Hulk', 'Exit Wounds']